<a href="https://colab.research.google.com/github/pluieciel/finetune_tools/blob/master/xlnet_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive
fp.write(s.encode('utf8'))

## Setup
Install dependencies

In [0]:
! pip install sentencepiece
! pip install pymupdf
! pip install pdfminer.six

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!rm -r predict_input
#!rm -r predict_input.zip

pdf to sentences

In [0]:
import shutil,os
import sys
sys.path.insert(1, '/content/drive/My Drive/xlnet')
import predict_tools

input_dir='/content/'
txt_sentence_dir='/content/predict_input/test/neg/'

r, d, f = next(os.walk(input_dir))
doc_list=[file for file in f if file.rfind('.pdf') != -1]

os.makedirs(txt_sentence_dir)
os.mkdir('/content/predict_input/test/pos/')
os.mkdir('/content/predict_input/test/other/')

for num, doc in enumerate(doc_list):
    input_pdf='/content/'+doc
    predict_tools.pdftosentence(input_pdf,txt_sentence_dir,num)

## Define Variables
Define all the dirs: data, xlnet scripts & pretrained model. 
If you would like to save models then you can authenticate a GCP account and use that for the OUTPUT_DIR & CHECKPOINT_DIR - you will need a large amount storage to fix these models. 

Alternatively it is easy to integrate a google drive account, checkout this guide for [I/O in colab](https://colab.research.google.com/notebooks/io.ipynb) but rememeber these will take up a large amount of storage. 


##Predict


In [0]:
#! unzip predict_input.zip

sentences to tsv

In [0]:
predict_command = "python drive/My\ Drive/xlnet/run_classifier_3.py \
  --do_predict=True \
  --predict_dir=predict_out3 \
  --task_name=transcript_classification \
  --overwrite_data=True \
  --data_dir=predict_input \
  --output_dir=out \
  --model_dir=drive/My\ Drive/xlmodel \
  --uncased=False \
  --spiece_model_file=drive/My\ Drive/xlmodel/spiece.model \
  --model_config_path=drive/My\ Drive/xlmodel/xlnet_config.json \
  --init_checkpoint=drive/My\ Drive/xlmodel/model.ckpt-2200 \
  --predict_ckpt=drive/My\ Drive/xlmodel/model.ckpt-2200 \
  --max_seq_length=64"

! {predict_command}


tsv to highlighted pdf

In [0]:
tsv='/content/predict_out3/transcript_classification.tsv'

for num, doc in enumerate(doc_list):
    input_pdf='/content/'+doc
    outdoc='/content/'+doc[0:-4]+'_MH2.pdf'

    predict_tools.addhighlight_3class(input_pdf,outdoc,predict_tools.tsvtolist_3class(tsv,txt_sentence_dir,num))